In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv("../input/nyc-crime-stats/NYC_crime.csv")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* Note this is very much a beginers look. Any help with  the CHI Square testing at the end would be apprciated. 

# Quick look at the data

Can see that this is very large and any cell takes a long time to complete. Therefore will split out one year to make it more manageable. I will concentrate on 2019

In [ ]:
df.describe()

In [ ]:
df.head()


In [ ]:
# create a column for year 
df['year'] = df['arrest_date'].map(lambda x: x[:4])

In [ ]:
# check
df.head()

In [ ]:
# split off year 2019
df_2019 = df[df['year'] == '2019']

# Look at how crime varies though the year
* first for NYC as a whole 
* second for the different boroughs

In [ ]:
# convert arrest_date column to date time 

df_2019['arrest_date'] =  pd.to_datetime(df_2019['arrest_date'], infer_datetime_format=True)

In [ ]:
# check 
df_2019.head()

In [ ]:
# group by arrest month and then get the size or each month for total number of crimes
NYC_crime_by_month_total_2019 = df_2019.groupby(df_2019['arrest_date'].dt.strftime('%m')).size()

# then get the total number of crimes for the year 
NYC_total_crime_2019 = NYC_crime_by_month_total_2019.values.sum()

# Use this figure to get the % of crime per month for NYC rounded to 2dp
NYC_crime_by_month_propotional_2019 = np.round(((NYC_crime_by_month_total_2019.values/NYC_total_crime_2019)*100),2)

# plot this in a graph 
fig = plt.figure(figsize=(8,4))
plt.plot(NYC_crime_by_month_total_2019.index, NYC_crime_by_month_propotional_2019)
plt.yticks(list(range(7,11)))
plt.legend(['2019'])

In [ ]:
df_2019_man = df_2019[df_2019['arrest_boro'] == 'M']
df_2019_bro = df_2019[df_2019['arrest_boro'] == 'K']
df_2019_que = df_2019[df_2019['arrest_boro'] == 'Q']
df_2019_bro = df_2019[df_2019['arrest_boro'] == 'B']
df_2019_sta = df_2019[df_2019['arrest_boro'] == 'S']

list_of_boros_df = [df_2019_man, df_2019_bro, df_2019_que, df_2019_bro, df_2019_sta]
list_of_boro_names = ['Manhatten', 'Brooklyn', 'Queens', 'Bronx', 'Statten Island']
prop_crim_boro_dict = {}

for boro, boro_name in zip(list_of_boros_df, list_of_boro_names):

    # group by arrest month and then get the size or each month for total number of crimes per boro
    boro_crime_by_month_total_2019 = boro.groupby(boro['arrest_date'].dt.strftime('%m')).size()

    # then get the total number of crimes for the year for the boro
    boro_total_crime_2019 = boro_crime_by_month_total_2019.values.sum()

    # Use this figure to get the % of crime per month for boro rounded to 2dp
    boro_crime_by_month_proportional_2019 = np.round(((boro_crime_by_month_total_2019.values/boro_total_crime_2019)*100),2)
    prop_crim_boro_dict[boro_name] = boro_crime_by_month_proportional_2019
    

for boro, crime in prop_crim_boro_dict.items():
    plt.plot(list(range(1, 13)), crime, label=boro)
plt.legend()

                         

# CHI Square Test Expected Monthly Crime Proportion

Looking at the graph above, all the boroughs look to have the same proportion of crime in each month with slight varience. High in January, low in December... 

I was wondering if we look a bit deeper into this, is this the same for all crimes. So if we split the df_2019 data frame into two. 
1. for crimes relating to drugs
2. for the rest of the crimes 

Then we like before plot these on a graph do they look different. But to be more precise, could we run a CHI Squared test to prove they are statistically signifcantly different. 

In [ ]:
df['ofns_desc'].unique()

In [ ]:
# looks like 'controlled substances' and 'drugs' should cover all drig offences
df_2019_drug_offences = df_2019[df_2019['ofns_desc'].str.contains('DRUG|CONTROLLED SUBSTANCES OFFENSES')]
df_2019_no_drug_offences = df_2019[df_2019['ofns_desc'].str.contains('DRUG|CONTROLLED SUBSTANCES OFFENSES')==False]

In [ ]:
# check 
print('non drug offences =', df_2019_no_drug_offences['ofns_desc'].nunique(), '\n','drug offences = ',  df_2019_drug_offences['ofns_desc'].nunique())

In [ ]:
# Create a function for the graphs 

def plot_graphs(df, title):
   # group by arrest month and then get the size or each month for total number of crimes
    NYC_crime_by_month_total_2019 = df.groupby(df['arrest_date'].dt.strftime('%m')).size()

    # then get the total number of crimes for the year 
    NYC_total_crime_2019 = NYC_crime_by_month_total_2019.values.sum()

    # Use this figure to get the % of crime per month for NYC rounded to 2dp
    NYC_crime_by_month_propotional_2019 = np.round(((NYC_crime_by_month_total_2019.values/NYC_total_crime_2019)),2)
    
    
    # plot this in a graph 
    fig = plt.figure(figsize=(8,4))
    plt.plot(NYC_crime_by_month_total_2019.index, NYC_crime_by_month_propotional_2019)
    plt.legend([title])
    
    return NYC_crime_by_month_total_2019, NYC_crime_by_month_propotional_2019

In [ ]:
# get total and proportion of drug offfences in NYC

total_drug_offences_per_month, prop_drug_offences_per_month = plot_graphs(df_2019_drug_offences, 'drug_offences')

print('total drug offences = ', total_drug_offences_per_month.values, '\n', 'proportion of drug offences = ', prop_drug_offences_per_month)

# get total and proportion of non drug offfences in NYC

total_non_drug_offences_per_month, prop_non_drug_offences_per_month = plot_graphs(df_2019_no_drug_offences, 'non_drug_offences')

print('total non drug offences = ', total_non_drug_offences_per_month.values, '\n', 'proportion of non drug offences = ', prop_non_drug_offences_per_month)

# Hypothosis Testing 

The null hypothesis is that the number of drug crimes in NYC by month:
[1723 1538 1652 1478 1374 1081 1307 1322 1161 1333 1161  820] 

is the same proportion of crime in NYC by month:
[9.87 8.29 9.07 8.76 8.91 7.97 8.86 8.26 7.82 8.16 7.38 6.66]


H1 is that there is a difference. 
We will use a signifcance of alpha 0.1%, ie the chance of getting the drug offence distribution is less than 0.1%



In [ ]:
#  what would be the expected number of drug crimes per month 

# total drug crimes in 2019

total_drug_crimes = total_drug_offences_per_month.values.sum()

# therefore what was the expected number of drug crimes per month 

expected_drug_crimes_per_month = prop_non_drug_offences_per_month * total_drug_crimes

expected_drug_crimes_per_month

In [ ]:
# CHI Square staistic then = (observed - expected)^2/expected 

difference_observed_expected = total_drug_offences_per_month.values - expected_drug_crimes_per_month

difference_squared = np.square(difference_observed_expected)

squ_difference_dived = difference_squared/expected_drug_crimes_per_month

CHI_stat = squ_difference_dived.sum()

print('CHI SQUARED Stat =', CHI_stat)

If we take the degrees of freedom to be 11 (12 months - 1) and alpha to be 0.1

gives a crtiical value of 18.

means we definately reject the null hypothesis and drug crimes vary differently to other crimes in the year. 